In [1]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

/bin/bash: nvidia-smi: command not found


In [7]:
save_to_gdrive = True #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [8]:
import os
import json
import pickle
import numpy as np

In [10]:
pwd

'/Users/chenxiluo/Downloads'

In [16]:
def get_json(file_path):
    return json.load(open(file_path))

path = '/Users/chenxiluo/Downloads/autocast/'
autocast_questions = get_json(path + 'autocast_questions.json')
#negated_tf_questions = get_json(path + 'negated_tf_questions.json')
#test_questions = get_json(path + 'autocast_competition_test_set.json')
#test_ids = [q['id'] for q in test_questions]
#print('autocast_questions', len(autocast_questions))
#print('negated_tf_questions', len(negated_tf_questions))
#print('test_questions', len(test_questions))
#autocast_questions = pd.read_csv('/Users/chenxiluo/Downloads/autocast_test_set_with_answers.csv')

## Create baseline models outputting random answers

In [17]:
def random_baseline_model(question):
    if question['qtype'] == 't/f':
        return np.random.random(size=2)
    elif question['qtype'] == 'mc':
        probs = np.random.random(size=len(question['choices']))
        return probs / probs.sum()
    elif question['qtype'] == 'num':
        return np.random.random()


def calibrated_random_baseline_model(question):
    if question['qtype'] == 't/f':
        pred_idx = np.argmax(np.random.random(size=2))
        pred = np.ones(2)
        pred[pred_idx] += 1e-5
        return pred / pred.sum()
    elif question['qtype'] == 'mc':
        pred_idx = np.argmax(np.random.random(size=len(question['choices'])))
        pred = np.ones(len(question['choices']))
        pred[pred_idx] += 1e-5
        return pred / pred.sum()
    elif question['qtype'] == 'num':
        return 0.5

## Get performance on the Autocast train set

Note that the Autocast dataset contains questions in the competition test set. Those should not be used.

In [18]:
def brier_score(probabilities, answer_probabilities):
    return ((probabilities - answer_probabilities) ** 2).sum() / 2

In [19]:
preds = []
answers = []
qtypes = []
for question in autocast_questions:
    if question['id'] in test_ids: # skipping questions in the competition test set
        continue
    if question['answer'] is None: # skipping questions without answer
        continue
    preds.append(calibrated_random_baseline_model(question))
    print(calibrated_random_baseline_model(question))
    if question['qtype'] == 't/f':
        ans_idx = 0 if question['answer'] == 'no' else 1
        ans = np.zeros(len(question['choices']))
        ans[ans_idx] = 1
        qtypes.append('t/f')
    elif question['qtype'] == 'mc':
        ans_idx = ord(question['answer']) - ord('A')
        ans = np.zeros(len(question['choices']))
        ans[ans_idx] = 1
        qtypes.append('mc')
    elif question['qtype'] == 'num':
        ans = float(question['answer'])
        qtypes.append('num')
    answers.append(ans)

NameError: name 'test_ids' is not defined

## Evaluate the model

In [16]:
tf_results, mc_results, num_results = [],[],[]
for p, a, qtype in zip(preds, answers, qtypes):
    if qtype == 't/f':
        tf_results.append(brier_score(p, a))
    elif qtype == 'mc':
        mc_results.append(brier_score(p, a))
    else:
        num_results.append(np.abs(p - a))

print(f"T/F: {np.mean(tf_results)*100:.2f}, MCQ: {np.mean(mc_results)*100:.2f}, NUM: {np.mean(num_results)*100:.2f}")
print(f"Combined Metric: {(np.mean(tf_results) + np.mean(mc_results) + np.mean(num_results))*100:.2f}")

T/F: 25.00, MCQ: 38.05, NUM: 22.63
Combined Metric: 85.67


## Make predictions on test set

In [11]:
preds = []
for question in test_questions:
    preds.append(calibrated_random_baseline_model(question))

In [12]:
if not os.path.exists('submission'):
    os.makedirs('submission')

with open(os.path.join('submission', 'predictions.pkl'), 'wb') as f:
    pickle.dump(preds, f, protocol=2)

!cd submission && zip ../submission.zip ./* && cd ..
print("here")

  adding: predictions.pkl (deflated 79%)
here


In [8]:
!ls

8843.pdf                           autocast-master
Ch4_Divide and_Conquer.pdf         autocast_test_set_with_answers.csv
FileZilla.app                      evaluation.ipynb
HeapSort.pdf                       introduction-to-algorithms-1-3.pdf
Heaps.pdf                          starter.ipynb
Lab 08 Transformers.ipynb          submission
MathBackGround-A.pdf               vim-8.1.0741
Pytorch_Tutorial_Part_1.ipynb      重要文件
Visual Studio Code.app             霍乱时期的爱情.pdf
autocast


In [32]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return {'input_ids': torch.tensor(self.data[index]['input_ids'], dtype=torch.long),
                'attention_mask': torch.tensor(self.data[index]['attention_mask'], dtype=torch.long)}


df = pd.read_csv('autocast_test_set_with_answers.csv')
data = df.to_dict(orient='records')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

tokenized_data = [{'input_ids': tokenizer.encode(row['text'], add_special_tokens=True),
                   'attention_mask': [1] * len(tokenizer.encode(row['text'], add_special_tokens=True))}
                  for row in data]

dataset = CustomDataset(tokenized_data)

dataloader = DataLoader(dataset, batch_size=32)

model.eval()

predictions = []

for batch in dataloader:
    with torch.no_grad():
        inputs = {'input_ids': batch['input_ids'],
                  'attention_mask': batch['attention_mask']}
        outputs = model(**inputs)
        logits = outputs[0]
        batch_predictions = torch.argmax(logits, dim=1).tolist()
        predictions += batch_predictions

print(predictions)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

KeyError: 'text'

In [12]:
# import packages
import numpy as np
import pandas as pd
import requests
import os
from sklearn.model_selection import train_test_split

# importing data
df = pd.read_csv('autocast_train_questions.csv')
test = []
for i in range(len(df)):
    if (df['status'][i] == 'Resolved'):
        test.append(True)
    else:
        test.append(False)

X = df.iloc[test]


# using the train test split function
X_train, X_test = train_test_split(X,random_state=104,test_size=0.3,shuffle=True)

for link in X_train['source']:
    if len(link) != 0:
        my_list = link[1:-1].split(', ')
        for l in my_list:
            url = l[1:-1]
            filename = url.replace('/', '-').replace(':', '_') + '.txt'
            print(filename)
            response = requests.get(l[1:-1], verify = False)
            if response.status_code == 200:
                with open(filename, 'wb') as file:
                    #print(file)
                    file.write(response.content)
            #break
                    
        print("Successfully wrote response to output.txt")
        break
    

https_--techcrunch.com-2021-01-25-blackberry-and-baidu-deepen-autonomous-connected-car-partnership-.txt
https_--www.lexology.com-library-detail.aspx?g=0b02f4ca-8c4f-4b66-aa4c-1ce5d6dfb97a.txt


/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techcrunch.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.lexology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https_--apnews.com-article-technology-driverless-cars-joint-ventures-las-vegas-nevada-99e57b67cd6685b33bb19efa7d5ff63e.txt


/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apnews.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https_--fortune.com-2021-05-28-germany-automobile-legalize-robotaxi-autonomous-vehicle-.txt
https_--www.econotimes.com-Baidu-to-make-thousands-of-robotaxis-for-a-driverless-service-in-China-1611110.txt


...

https_--www.autoweek.com-news-technology-a36354137-beijing-olympics-robotaxi-china-.txt
https_--hypebeast.com-2021-2-china-autox-fully-driverless-autonomous-taxi-launch.txt


/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.autoweek.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hypebeast.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https_--vnexplorer.net-robotaxi-race-heats-up-in-china-autox-unveils-superior-upgrade-2-a2021249509.html.txt


/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vnexplorer.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vnexplorer.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Successfully wrote response to output.txt


In [88]:
import pandas as pd
# Create a sample DataFrame
df = pd.DataFrame({
    'Name': ['John', 'Alice', 'Bob'],
    'Age': [25, 30, 35],
    'City': ['New York', 'Paris', 'London']
})

# Select a subset of rows and columns using the loc[] method
subset = df.loc[1:, ['Name']]
print(subset)

    Name
1  Alice
2    Bob


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Format the data
data = [
    {"question": "What will the end-of-day closing value for the dollar against the renminbi be on 1 January 2016?", "choices": ["A. London", "B. Berlin", "C. Paris", "D. Rome"], "answer": "C"},
    {"question": "What is the largest planet in the solar system?", "choices": ["A. Earth", "B. Venus", "C. Jupiter", "D. Saturn"], "answer": "C"}
]

# Tokenize the data
tokenized_data = []
for item in data:
    question = f"question: {item['question']} "
    choices = " ".join([f"{choice}" for choice in item['choices']])
    answer = f"answer: {item['answer']}"
    text = question + choices + answer
    encoded_input = tokenizer(text, return_tensors='pt')
    tokenized_data.append(encoded_input)
    print(question)
    print(choices)
    print(answer)

question: What is the capital of France? 
A. London B. Berlin C. Paris D. Rome
answer: C
question: What is the largest planet in the solar system? 
A. Earth B. Venus C. Jupiter D. Saturn
answer: C


In [4]:
import requests

# Define a list of website URLs
urls = ['https://www.example.com', 'https://www.google.com', 'https://www.github.com']

# Loop through the list of URLs
for url in urls:
    # Make a GET request to the website
    response = requests.get(url)
    
    # Get the content of the response
    content = response.content
    
    # Create a new text file and write the content to it
    with open(f"{url.split('//')[1]}.txt", 'wb') as file:
        file.write(content)

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
text = "Your text here."
encoded_input = tokenizer(text, return_tensors='pt')
from transformers import Trainer, TrainingArguments

model_path = "/Users/chenxiluo/Downloads"

training_args = TrainingArguments(
    output_dir=model_path, 
    num_train_epochs=1, 
    per_device_train_batch_size=1, 
    save_steps=1000
)

trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=encoded_input
)

trainer.train()
model.save_pretrained(model_path)
generated_text = model.generate(
    input_ids=encoded_input["input_ids"], 
    max_length=100, 
    temperature=0.7
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))


/opt/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,past_key_values,attention_mask,token_type_ids,position_ids,head_mask,inputs_embeds,encoder_hidden_states,encoder_attention_mask,labels,use_cache,output_attentions,output_hidden_states,return_dict,labels,label_ids,label.

In [1]:
!pwd

/Users/chenxiluo/Downloads


In [8]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Preprocess the data
input_text = "What is the capital of France?"
source_link = "https://en.wikipedia.org/wiki/Paris"
answer_options = ["Paris", "Berlin", "Madrid", "Rome"]
correct_answer = 0 # the correct answer is the first one
input_ids = tokenizer.encode(input_text, return_tensors='pt')
source_ids = tokenizer.encode(source_link, return_tensors='pt')
option_ids = [tokenizer.encode(option, return_tensors='pt') for option in answer_options]

# Create a new labels tensor with the correct batch size
batch_size = input_ids.shape[0]
labels = torch.cat([source_ids] * batch_size)
print(input_ids)
# Fine-tune the model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for i in range(1000):
    loss, _ = model(input_ids, labels=labels)
    for option_id in option_ids:
        loss += model(option_id, labels=option_id)[0] # add loss for each option
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

# Evaluate the model
test_text = "What is the tallest mountain in the world?"
test_link = "https://en.wikipedia.org/wiki/Mount_Everest"
test_options = ["Mount Everest", "K2", "Makalu", "Lhotse"]
test_ids = tokenizer.encode(test_text, return_tensors='pt')
test_option_ids = [tokenizer.encode(option, return_tensors='pt') for option in test_options]

# Create a new labels tensor with the correct batch size
batch_size = test_ids.shape[0]
test_labels = torch.cat([source_ids] * batch_size)

test_loss, _ = model(test_ids, labels=test_labels)
for test_option_id in test_option_ids:
    test_loss += model(test_option_id, labels=test_option_id)[0] # add loss for each option
test_answer_index = torch.argmax(test_loss)
test_answer = test_options[test_answer_index]
print(test_answer)


tensor([[2061,  318,  262, 3139,  286, 4881,   30]])


ValueError: Expected input batch_size (6) to match target batch_size (10).

In [14]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Load and preprocess the data
with open('multiple_choice_dataset.txt', 'r') as f:
    dataset = f.readlines()

input_ids = []
output_ids = []

for line in dataset:
    question, options = line.split('\n')[0].split('?')
    options = options.split(', ')
    input_text = question + '?'
    output_text = ''
    for i, option in enumerate(options):
        output_text += f'{chr(65+i)}. {option}\n'
    input_ids.append(tokenizer.encode(input_text, add_special_tokens=False, return_tensors='pt'))
    output_ids.append(tokenizer.encode(output_text, add_special_tokens=False, return_tensors='pt'))

# Fine-tune the model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for i in range(1000):
    total_loss = 0
    for input_id, output_id in zip(input_ids, output_ids):
        loss, _ = model(input_id, labels=output_id)
        total_loss += loss
    total_loss.backward()
    optimizer.step()
    optimizer.zero_grad()

# Generate answers
test_text = "What is the capital of France?"
test_input_ids = tokenizer.encode(test_text, add_special_tokens=False, return_tensors='pt')
output_text = tokenizer.decode(model.generate(test_input_ids, max_length=128, do_sample=True))
options = [option.strip() for option in output_text.split('\n') if option.strip()]
print(options)


ValueError: Expected input batch_size (6) to match target batch_size (17).

In [15]:
from transformers import AutoTokenizer, TFGPT2Model
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = TFGPT2Model.from_pretrained("gpt2")

inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
outputs = model(inputs)

last_hidden_states = outputs.last_hidden_state
print()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2023-04-29 17:31:57.860571: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 17:31:57.926281: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.
